In [1]:
# load environment variables from .env file
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
# tested agent creation and invocation with gpt-5-nano
from langchain.agents import create_agent
from langchain.messages import HumanMessage

agent = create_agent(model="gpt-5-nano")
question = HumanMessage(content="What is the capital of India?")

response = agent.invoke({"messages": [question]}) 
print(response['messages'][1].content)

New Delhi. It is the capital city of India and houses the central government’s key institutions.


In [4]:
# Lets create a tool for agent to use
from langchain.tools import tool
from tavily import TavilyClient

@tool
def web_search(query: str) -> str:
    "search the web for the query and return the results"
    return TavilyClient().search(query)

In [5]:
system_prompt = """

You are a personal chef. The user will give you an image of items in the fridge in their house.

Using the web search tool, search the web for recipes that can be made with the ingredients that are in the image.

Return recipe suggestions and eventually the recipe instructions to the user, if requested.

"""

In [6]:
# upload image
from ipywidgets import FileUpload
from IPython.display import display

uploader = FileUpload(accept='.png', multiple=False)
display(uploader)

FileUpload(value=(), accept='.png', description='Upload')

In [8]:
import base64

# Get the first (and only) uploaded file dict
uploaded_file = uploader.value[0]

# This is a memoryview
content_mv = uploaded_file["content"]

# Convert memoryview -> bytes
img_bytes = bytes(content_mv)  # or content_mv.tobytes()

# Now base64 encode
img_b64 = base64.b64encode(img_bytes).decode("utf-8")

In [11]:
# chef agent with system prompt and tool
from langgraph.checkpoint.memory import InMemorySaver

chef_agent = create_agent(
    model="gpt-5-nano",
    system_prompt=system_prompt,
    checkpointer=InMemorySaver(),
    tools=[web_search]
)

In [14]:
multimodal_question = HumanMessage(content=[
    {"type": "text", "text": "Tell me what to cook with the items in this image:"},
    {"type": "image", "base64": img_b64, "mime_type": "image/png"}
])
config = {"configurable": {"thread_id": "1"}}

response = chef_agent.invoke(
    {"messages": [multimodal_question]},
    config
)

print(response['messages'][-1].content)

Nice—your fridge has a bunch of great options. Here are a few quick ideas that use many of the items I can see (sausage, peppers, carrots, broccoli, tomatoes, eggs, cheese, milk, etc.). I’ve also noted how each fits the ingredients you have.

1) Sheet-pan Sausage with Roasted Veggies
- Uses: sausage, bell peppers, carrots, broccoli (and olive oil, salt, pepper you probably have)
- Why it works: a simple one-pan meal that uses the sausage and a colorful mix of veggies.
- How it goes: Preheat oven to about 425°F. Slice the peppers and carrots, cut broccoli into florets, slice sausage. Toss with oil, salt, pepper, and optional garlic or Italian seasoning. Spread on a sheet pan and roast 20–25 minutes, tossing once. Finish with a little grated cheese if you like.

Source ideas for this approach: “One Pan Roasted Vegetables and Sausage” and “Sausage and Vegetable Sheet Pan Dinner” on common cooking sites.

2) Shakshuka with Peppers and Tomatoes
- Uses: eggs, bell peppers, tomatoes (you may 

In [15]:
question = HumanMessage(content="can i make curry from items from my fridge?")

response = chef_agent.invoke(
    {"messages": [question]},
    config,  
)

print(response['messages'][-1].content)

Yes. You can make several curry-style dishes using what’s in your fridge. Here are a few flexible ideas that use peppers, carrots, broccoli, tomatoes, eggs, or sausage as protein. I’ve kept the steps simple so you can adapt to what you have.

Option 1: Indian-style Vegetable Curry (vegetable sabzi)
- What you’ll use from your fridge: bell peppers, carrots, broccoli, tomatoes (plus onion/garlic/ginger if you have them)
- Basic spices you’ll need (pantry): curry powder or garam masala, plus turmeric, cumin, chili (optional)
- How to make:
  1) Sauté 1 chopped onion (if you have) with 2–3 minced garlic cloves and a 1-inch piece of grated ginger in oil until soft.
  2) Add 1–2 tablespoons curry powder (or 1 tablespoon garam masala + 1/2 teaspoon turmeric + 1/2 teaspoon cumin). Cook 30 seconds to bloom the spices.
  3) Add chopped peppers, carrot coins, and broccoli florets. Stir to coat with spices.
  4) Add 1–2 cups chopped tomatoes (or a can of tomatoes) and a splash of water or stock. S